In [1]:
# ejercicio de regresion lineal

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Regresion Lineal 01').getOrCreate()

In [4]:
publicidadDF = spark.read.format('csv').options(header='true',inferschema='true').\
load("file:///c:/archivos/publicidad.csv")

In [5]:
publicidadDF.printSchema()

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)



In [6]:
publicidadDF.show(10)

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
|  8.7| 48.9|     75.0|  7.2|
| 57.5| 32.8|     23.5| 11.8|
|120.2| 19.6|     11.6| 13.2|
|  8.6|  2.1|      1.0|  4.8|
|199.8|  2.6|     21.2| 10.6|
+-----+-----+---------+-----+
only showing top 10 rows



In [7]:
# convertir la data a un vector denso
# caracteristicas :-> TV,Radio, NewsPapers
# label :-> Sales

In [8]:
from pyspark.ml.linalg import Vectors 

In [12]:
def transformarDatos(data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])

In [13]:
publicidadTransformadaDF = transformarDatos(publicidadDF)

In [16]:
publicidadTransformadaDF.show(3)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[230.1,37.8,69.2]| 22.1|
| [44.5,39.3,45.1]| 10.4|
| [17.2,45.9,69.3]|  9.3|
+-----------------+-----+
only showing top 3 rows



In [26]:
from pyspark.ml.feature import VectorIndexer
indexadorFeature01 = VectorIndexer(inputCol='features',outputCol='featuresIndexadas',maxCategories=1000).\
fit(publicidadTransformadaDF)

In [30]:
from pyspark.ml.regression import LinearRegression

In [31]:
rl = LinearRegression()

In [ ]:
rl.fit(datosTrain)

In [34]:
(datosTrain,datosTest) = publicidadTransformadaDF.randomSplit([0.7,0.3])

In [32]:
from pyspark.ml import Pipeline

In [35]:
pipeline = Pipeline(stages=[indexadorFeature01,rl])
modeloRL = pipeline.fit(datosTrain)

In [38]:
modeloRL.stages

[VectorIndexerModel: uid=VectorIndexer_c65b5a48e702, numFeatures=3, handleInvalid=error,
 LinearRegressionModel: uid=LinearRegression_af048eb0acfd, numFeatures=3]

In [37]:
modeloRL.stages[-1].coefficients

DenseVector([0.045, 0.195, -0.003])

In [39]:
prediccionesDF = modeloRL.transform(datosTest)

In [40]:
prediccionesDF.show()

+-----------------+-----+------------------+------------------+
|         features|label| featuresIndexadas|        prediction|
+-----------------+-----+------------------+------------------+
|   [4.1,11.6,5.7]|  3.2|   [1.0,43.0,16.0]| 5.369494700184046|
| [13.2,15.9,49.6]|  5.6| [10.0,58.0,133.0]| 6.484630739785105|
| [16.9,43.7,89.4]|  8.7|[11.0,151.0,169.0]| 11.95258858488634|
| [17.2,45.9,69.3]|  9.3|[12.0,156.0,160.0]|12.456158174553375|
| [26.8,33.0,19.3]|  8.8| [22.0,113.0,59.0]|10.523949513141162|
|  [27.5,1.6,20.7]|  6.9|   [23.0,7.0,66.0]| 4.426925285775773|
| [38.0,40.3,11.9]| 10.9| [27.0,139.0,38.0]|12.474362142355707|
|  [38.2,3.7,13.8]|  7.6|  [28.0,18.0,44.0]| 5.339100548231294|
|  [48.3,47.0,8.5]| 11.6| [34.0,160.0,26.0]| 14.25510260412803|
|[67.8,36.6,114.0]| 12.5|[43.0,126.0,171.0]|12.784420368999983|
| [68.4,44.5,35.6]| 13.6|[44.0,154.0,105.0]|14.590085834508587|
| [69.2,20.5,18.3]| 11.3|  [46.0,72.0,55.0]| 9.997594460264432|
| [76.4,26.7,22.3]| 11.8|  [54.0,90.0,71